# Numerically Showing Unimodality

Randomly generate with random support points, uniform, and Normal, all 20 support points. For uniform, draw a, b from a random distribution, as well as for Normal. Try for 2, 3, and 4-node distributions. 

To check unimodality, sample from lambdas [-2, 2], which by Chebyshev's inequality must cover 3/4 of the probability mass of the demand distributions. We check for lambda_time, so there's only one parameter to learn. We check in increments of 0.01, and establish unimodality on our checked points.

In [4]:
import sys
import random
import os
import random

# add library to path (or else, src not visible)
sys.path.insert(0, "../../")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

from src.AllocationSolver import AllocationSolver, lambda_bound
from src.dists import SymmetricDiscreteDistribution, Distribution, UniformDistribution, NormalDistribution
from src.monte_carlo import hoeffding_bound
from src.format_data import clean_data
from src.random_problem import generate_random_problem, generate_general_distribution, generate_normal_distribution, generate_uniform_distribution

# use seaborn style plotting for paper
sns.set_context("paper")
sns.set_style("whitegrid")

In [5]:
def is_unimodal_sequence(sequence, tol=1e-4):
    """
    Check if a sequence is unimodal

    That is, check if left of the maximum, the sequence is increasing
    and right of the maximum, the sequence is decreasing
    """
    max_index = sequence.index(max(sequence))
    return all(-tol <= sequence[i+1] - sequence[i] for i in range(max_index)) and all(sequence[i] - sequence[i+1] >= -tol for i in range(max_index, len(sequence)-1))

## Uniform and Normal Distributions

In [9]:
random.seed(0)
np.random.seed(0)

num_problems = 1000

for _ in tqdm(range(num_problems)):
    # prob = generate_random_problem(3, generate_normal_distribution((1, 2), random.random() * 0.5, n=5), allocation_method="lambda_time")
    prob = generate_random_problem(3, generate_uniform_distribution((1, 2), 10))
    
    # for alphas = [0.1, 0.2, 0.5, 1, 2, 5, 10]
    for a in [0.1, 0.2, 0.5, 1, 2, 5, 10, np.inf]:
        prob.change_alpha(a)
        # for supply scarcity = [0.1, 0.2 ... 1.9, 2]
        for ss in np.arange(0.1, 2, 0.1):
            prob.change_initial_supply(sum(prob.demand_means) * ss)
            Z_values = []

            # for lambdas = [-2, -1.9, -1.8 ... 1.9, 2]
            for l in np.arange(prob.get_lambda_range()[0], 2, 0.1):
                prob.set_lambda_schedule([l for _ in range(prob.N)])
                Z = prob.solve()[0]
                if Z > 0:
                    Z_values.append(Z)

            if not is_unimodal_sequence(Z_values):
                # print formatted table of lambdas and Z values
                for l, Z in zip(np.arange(-2, 2, 0.1), Z_values):
                    print(f"Lambda: {l:.1f}, Z: {Z:.5f}")
                print(prob)
                print(f"Alpha: {a}, Supply Scarcity: {ss}")
                print(lambda_bound(prob.demand_distributions, lambda_time=True))

  0%|          | 0/1 [00:00<?, ?it/s]

Lambda: -2.0, Z: 0.18306
Lambda: -1.9, Z: 0.07523
Lambda: -1.8, Z: 0.37448
Lambda: -1.7, Z: 0.00000
Lambda: -1.6, Z: 0.01763
Lambda: -1.5, Z: 0.03397
Lambda: -1.4, Z: 0.04817
Lambda: -1.3, Z: 0.06008
Lambda: -1.2, Z: 0.06984
Lambda: -1.1, Z: 0.07768
Lambda: -1.0, Z: 0.08388
Lambda: -0.9, Z: 0.08869
Lambda: -0.8, Z: 0.09235
Lambda: -0.7, Z: 0.09505
Lambda: -0.6, Z: 0.09698
Lambda: -0.5, Z: 0.09826
Lambda: -0.4, Z: 0.09902
Lambda: -0.3, Z: 0.09935
Lambda: -0.2, Z: 0.09934
Lambda: -0.1, Z: 0.09905
Lambda: 0.0, Z: 0.09853
Lambda: 0.1, Z: 0.09784
Lambda: 0.2, Z: 0.09700
Lambda: 0.3, Z: 0.09605
Lambda: 0.4, Z: 0.09501
Lambda: 0.5, Z: 0.09391
Lambda: 0.6, Z: 0.09276
Lambda: 0.7, Z: 0.09157
Lambda: 0.8, Z: 0.09035
Lambda: 0.9, Z: 0.08912
Lambda: 1.0, Z: 0.08788
Lambda: 1.1, Z: 0.08665
Lambda: 1.2, Z: 0.08541
Lambda: 1.3, Z: 0.08418
Lambda: 1.4, Z: 0.08297
Lambda: 1.5, Z: 0.08176
Lambda: 1.6, Z: 0.08058
Lambda: 1.7, Z: 0.07941
Node 0: { 1: 0.5, 2: 0.5 }
Node 1: { 1: 0.5, 2: 0.5 }
Node 2: { 1: 0

  0%|          | 0/1 [00:05<?, ?it/s]

Lambda: -2.0, Z: 0.55458
Lambda: -1.9, Z: 0.59532
Lambda: -1.8, Z: 0.58745
Lambda: -1.7, Z: 0.05523
Lambda: -1.6, Z: 0.09361
Lambda: -1.5, Z: 0.12677
Lambda: -1.4, Z: 0.15477
Lambda: -1.3, Z: 0.18111
Lambda: -1.2, Z: 0.21435
Lambda: -1.1, Z: 0.25000
Lambda: -1.0, Z: 0.27936
Lambda: -0.9, Z: 0.31072
Lambda: -0.8, Z: 0.33551
Lambda: -0.7, Z: 0.35476
Lambda: -0.6, Z: 0.36939
Lambda: -0.5, Z: 0.38022
Lambda: -0.4, Z: 0.38791
Lambda: -0.3, Z: 0.39303
Lambda: -0.2, Z: 0.39607
Lambda: -0.1, Z: 0.39740
Lambda: 0.0, Z: 0.39735
Lambda: 0.1, Z: 0.39619
Lambda: 0.2, Z: 0.39408
Lambda: 0.3, Z: 0.39125
Lambda: 0.4, Z: 0.38785
Lambda: 0.5, Z: 0.38400
Lambda: 0.6, Z: 0.37977
Lambda: 0.7, Z: 0.37528
Lambda: 0.8, Z: 0.37058
Lambda: 0.9, Z: 0.36573
Lambda: 1.0, Z: 0.36080
Lambda: 1.1, Z: 0.35581
Lambda: 1.2, Z: 0.35077
Lambda: 1.3, Z: 0.34573
Lambda: 1.4, Z: 0.34073
Lambda: 1.5, Z: 0.33575
Lambda: 1.6, Z: 0.33082
Lambda: 1.7, Z: 0.32595
Lambda: 1.8, Z: 0.32116
Lambda: 1.9, Z: 0.31644
Node 0: { 1: 0.5, 2:

KeyboardInterrupt: 